# 09 - Disk Fragmenter

https://adventofcode.com/2024/day/9


In [42]:
// Read Input
const file = await Deno.readTextFile("input-base.txt");

const diskMap = []
for (let i = 0, id = 0; i < file.length; i += 2) {
    diskMap.push(
        {
            blocks: parseInt(file[i]),
            freeSpace: parseInt(file[i + 1]) || 0,
            id: id++
        }
    )
}
[diskMap.length, diskMap]

[
  10,
  [
    { blocks: 2, freeSpace: 3, id: 0 },
    { blocks: 3, freeSpace: 3, id: 1 },
    { blocks: 1, freeSpace: 3, id: 2 },
    { blocks: 3, freeSpace: 1, id: 3 },
    { blocks: 2, freeSpace: 1, id: 4 },
    { blocks: 4, freeSpace: 1, id: 5 },
    { blocks: 4, freeSpace: 1, id: 6 },
    { blocks: 3, freeSpace: 1, id: 7 },
    { blocks: 4, freeSpace: 0, id: 8 },
    { blocks: 2, freeSpace: 0, id: 9 }
  ]
]

In [43]:
// Prepare Data
const uncompressedDisk = diskMap.reduce((acc, { blocks, freeSpace }, i) => {
    return acc.concat(Array(blocks).fill(i), Array(freeSpace).fill('.'));
}, [])
uncompressedDisk.join('');

"00...111...2...333.44.5555.6666.777.888899"

In [44]:
// Compact Data

const compactIt = (disk) => {
    const compactDisk = [...disk]
    for (let i = 0, j = compactDisk.length - 1; i <= j; i += 1) {
        while (compactDisk[j] === '.' && j > i) j -= 1;
        if (compactDisk[i] === '.') {
            compactDisk[i] = compactDisk[j];
            compactDisk[j] = '.';
            j -= 1;
        }
    }
    return compactDisk;
}
compactIt(uncompressedDisk)

[
  0,   0,   9,   9,   8,   1,   1,   1,   8,   8,
  8,   2,   7,   7,   7,   3,   3,   3,   6,   4,
  4,   6,   5,   5,   5,   5,   6,   6,   ".", ".",
  ".", ".", ".", ".", ".", ".", ".", ".", ".", ".",
  ".", "."
]

In [45]:
// Part 1 - What is the resulting filesystem checksum ?

const checkSum = (disk) => {
    let sum = 0;
    for (let i = 0; i < disk.length; i += 1) {
        if (disk[i] === '.') break;
        sum += disk[i] * i;
    }
    return sum;
}
checkSum(compactIt(uncompressedDisk));

1928

In [46]:
// Part 2 - What is the resulting filesystem checksum with new method ?

import objects from "../../utils/objects.ts";

const compactItV2 = (disk: [{ blocks: number, id: number, freeSpace: number }]) => {
    const modifiedDisk = objects.deepCopy(disk);
    for (let i = modifiedDisk.length - 1; i > 0; i -= 1) {
        const rightMostBlockLength = modifiedDisk[i].blocks;
        for (let j = 0; j < i; j += 1) {
            if (modifiedDisk[j].freeSpace >= rightMostBlockLength) {
                modifiedDisk[j].freeSpace -= rightMostBlockLength;
                if (!modifiedDisk[j].welcomedBlocks) {
                    modifiedDisk[j].welcomedBlocks = []
                }
                modifiedDisk[j].welcomedBlocks.push(modifiedDisk[i]);
                modifiedDisk[i].moved = true
                break;
            }
        }
    }
    const compactDisk = modifiedDisk.reduce((acc, { blocks, freeSpace, welcomedBlocks, moved, id }) => {
        if (moved && !welcomedBlocks) acc.concat(Array(blocks + freeSpace).fill('.'));
        return acc.concat([
            ...Array(blocks).fill(moved ? '.' : id),
            ...((welcomedBlocks|| []).reduce((acc, { blocks, id }) => acc.concat(Array(blocks).fill(id)), [])),
            ...Array(freeSpace).fill('.')
        ])
    }, [])
    return compactDisk;
}

compactItV2(diskMap).reduce((sum, val, i) => val === '.' ? sum : sum + val * i, 0);

2858